# 데일리 항공편 (인천공항 입국) 정보 등록 자동화

In [1]:
#!pip selenium
import datetime as dt, pandas as pd, numpy as np
import os, time, glob, shutil, pymysql

from openpyxl import Workbook, load_workbook
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# URL
url = 'https://www.airportal.go.kr/life/airinfo/FlightScheduleToExcel.jsp'
now = time

# 아이디 비밀번호
site_id = "아이디"
site_pw = "비밀번호"

In [4]:
def get_flight(start_date,end_date) :
    import calendar
    
    # 웹드라이버 열기
    driver = webdriver.Chrome(r"{}\chromedriver.exe".format(os.getcwd()))
    driver.get(url)
    
    def get_path(path) : 
        return driver.find_element(by = By.CSS_SELECTOR, value = path)
    
    # 날짜 지정
    #rct3 = pd.date_range(start=yesterday.strftime('%Y%m%d'), end=yesterday.strftime('%Y%m%d'))
    rct3 = pd.date_range(start=start_date, end=end_date)
    dt_list = rct3.strftime("%Y%m%d").to_list()

    start = time.time()  # 시작 시간 저장


    # 날짜 검색창 Xpath
    xpath_text = '//*[@id="dateStart"]/div/input'
    # 검색하기 버튼
    xpath_search = '//*[@id="realContents"]/div/div[2]/div[1]/div[1]/a[2]'
    # 다운로드 버튼
    xpath_download = '//*[@id="realContents"]/div/div[2]/div[1]/div[1]/a[1]'
    # 출발 체크
    xpath_arrival = '//*[@id="realContents"]/div/div[2]/div[1]/div[2]/div/div/div[1]/label[2]/div/ins'
    try:
        driver.find_element(by = By.XPATH, value = '//*[@id="layer_popup_info_1"]/div[1]/a').click()
        time.sleep(1)
        driver.find_element(by = By.XPATH, value = '//*[@id="layer_popup_info_0"]/div[1]/a').click()
    except:
        print("로그인 창이 없습니다.")

    # 로그인 확인!!
    xpath_id = "/html/body/form/table[3]/tbody/tr[2]/td[2]/table[3]/tbody/tr/td[1]/table/tbody/tr[2]/td[2]/table/tbody/tr/td[1]/table[1]/tbody/tr/td[3]/input"
    xpath_pw = '/html/body/form/table[3]/tbody/tr[2]/td[2]/table[3]/tbody/tr/td[1]/table/tbody/tr[2]/td[2]/table/tbody/tr/td[1]/table[2]/tbody/tr/td[3]/input'
    xpath_login = '/html/body/form/table[3]/tbody/tr[2]/td[2]/table[3]/tbody/tr/td[1]/table/tbody/tr[2]/td[2]/table/tbody/tr/td[2]/input'
    driver.find_element(by = By.XPATH, value =xpath_id).send_keys('{}'.format(site_id)) 
    driver.find_element(by = By.XPATH, value =xpath_pw).send_keys('{}'.format(site_pw))

    # 임시로 생성한 아이디, 비밀번호
    driver.find_element(by = By.XPATH, value =xpath_login).click()

    ## 데이터 xpath
    #xpath_data = '//*[@id="mySheet-table"]/tbody/tr[3]/td/div/div[1]/table/tbody/tr[101]/td[14]'

    date_check = '#mySheet-table > tbody > tr:nth-child(3) > td > div > div.GMPageFirst > table > tbody > tr:nth-child(2) > td.GMClassReadOnly.GMWrap0.GMAlignCenter.GMText.GMCell.IBSheetFont0.HideCol0C2'
    
    # 날짜 지정 및 파일 다운로드
    print(int(start_date/100), int(end_date/100)+1)
    for month in range(int(start_date/100), int(end_date/100)+1) :
        print(month)
        get_path("#dateStart > div > input").click()
        get_path("#dateStart > div > input").clear()
        get_path("#dateStart > div > input").send_keys(month)
        get_path("#dateStart > div > input").send_keys('01')
        get_path("#dateStart > div > input").send_keys(Keys.RETURN)
        get_path("#dateStart > div > input").send_keys(Keys.BACKSPACE *8)
        get_path("#dateStart > div > input").send_keys(month)
        get_path("#dateStart > div > input").send_keys('01')
        get_path("#dateStart > div > input").send_keys(Keys.TAB)

        get_path("#dateEnd > div > input").send_keys(month)
        get_path("#dateEnd > div > input").send_keys(Keys.RETURN)
        get_path("#dateEnd > div > input").send_keys(Keys.BACKSPACE *2)
        get_path("#dateEnd > div > input").send_keys(str(calendar.monthrange(2022, int(str(month)[4:]))[1]))
        get_path("#dateEnd > div > input").send_keys(Keys.BACKSPACE *8)
        get_path("#dateEnd > div > input").send_keys(month)
        get_path("#dateEnd > div > input").send_keys(str(calendar.monthrange(2022, int(str(month)[4:]))[1]))
        get_path("#realContents > div > div.conLy > div.SearchArea01 > div.mainSearchBtnArea > a.mainSearchBtn").click()
        time.sleep(5)
        get_path("#realContents > div > div.conLy > div.SearchArea01 > div.mainSearchBtnArea > a.mainExcellBtn").click()
        time.sleep(10)
    time.sleep(15)
    #return '완료'

In [ ]:
#형식 -> 시작 날짜, 종료 날짜
get_flight(20221101, 20221231)

In [6]:
# 오늘 날짜 폴더 추가
os.mkdir("{}/Downloads/항공편_{}".format("/".join(os.getcwd().split("\\")[:-2]),dt.datetime.today().strftime('%m%d')))

In [10]:
# 다운된 파일 경로 변경
file_source = r"{}/Downloads/".format("/".join(os.getcwd().split("\\")[:-2]))
file_destination =r'{}/Downloads/항공편_{}/'.format("/".join(os.getcwd().split("\\")[:-2]),dt.datetime.today().strftime('%m%d'))

for file in Path(file_source).glob("항공기출도착현황*.xlsx"):
    shutil.move(os.path.join(file_source,file),file_destination)


# 파일 불러오기
try:
    path = r'{}/Downloads/항공편_{}/'.format("/".join(os.getcwd().split("\\")[:-2]),dt.datetime.today().strftime('%m%d'))
    files = glob.glob(path + "항공기출도착현황*.xlsx")
    excel = pd.DataFrame()
    for file_name in files:
        df = pd.read_excel(file_name)
        excel = excel.append(df, ignore_index=True)

except Exception as ex:
    print('error' + str(ex))
    
excel = excel[~excel['출발공항명'].isin(['제주','김해','대구','김포'])].drop_duplicates()
excel = excel[excel['구분'] == '여객'].sort_values(['날짜','계획시간'])

c:\Users\REDTABLE\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
conn = pymysql.connect(host='주소', user='db 사용자', password='비밀번호',
                       autocommit=True,cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()


# 공항코드 -> 국가 매칭
start = time.time()
sql = """
SELECT DISTINCT A.AR_CODE , A.NATION
FROM redtable_raw_2022.AIR_PORTAL A
"""
cursor.execute(sql)
na_dic = dict(pd.DataFrame(cursor.fetchall()).values)


# 공항코드 -> 대륙 매칭
start = time.time()
sql = """
SELECT DISTINCT A.AR_CODE , A.CONTINENT
FROM redtable_raw_2022.AIR_PORTAL A
"""
cursor.execute(sql)
co_dic = dict(pd.DataFrame(cursor.fetchall()).values)

# 편당 탑승인원
start = time.time()
sql = """
SELECT F.NTN_NM , F.AIR_TOURIST_RT
FROM redtable_raw_2022.FOREIGN_VISITOR F
WHERE F.AIR_DT = (SELECT max(F.AIR_DT)
FROM redtable_raw_2022.FOREIGN_VISITOR F)
"""
cursor.execute(sql)
rate_dic = dict(pd.DataFrame(cursor.fetchall()).values)

conn.close()
cursor.close()

print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간   


excel['출발국가']   = excel['출발공항코드'].apply(lambda x : na_dic.get(x))
excel['출발대륙']   = excel['출발공항코드'].apply(lambda x : co_dic.get(x))
excel['편당탑승객'] = excel['출발국가'].apply(lambda x : rate_dic.get(x))
excel = excel.sort_values(['날짜','계획시간'],ascending=[True,True])
excel['년월'] = excel['날짜'].apply(lambda x : int(str(x)[:6]))
og = excel.iloc[:,:-2].copy()

쿼리 완료 
쿼리 작동 시간 : 0.00899958610534668


## db에 항공 정보 등록

In [13]:
conn = pymysql.connect(host='주소', user='db 사용자', password='비밀번호',
                       autocommit=True,cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()
start = time.time()

# 중복 날짜 삭제
sql = """
DELETE
FROM redtable_raw_2022.AIR_PORTAL
WHERE DATE IN {}
""".format(tuple(og['날짜'].unique()))
cursor.execute(sql)

# 중복 날짜 조회
sql = """
SELECT *
FROM redtable_raw_2022.AIR_PORTAL
WHERE DATE IN {}
""".format(tuple(og['날짜'].unique()))
cursor.execute(sql)
result = pd.DataFrame(cursor.fetchall())
conn.close()
cursor.close()
result.head()
print('쿼리 완료', "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간   

# AIR_PORTAL DB에 저장
def save_to_AIR_PORTAL(data) :
    db_cols = 'AIR_AR_DP,DATE,AIR,AIR_NUM,AR_CODE,AR_AIRPORT_NM,DP_CODE,DP_AIRPORT_NM,EX_TIME,EX1_TIME,DP_TIME,OPTION1,LIVE,NATION,CONTINENT'
    df_result = data.drop_duplicates().fillna(np.nan).replace(["\\N"], [None]).replace([np.nan], [None])
    conn = pymysql.connect(host='주소', user='db 사용자', password='비밀번호',
                       autocommit=True,cursorclass=pymysql.cursors.DictCursor
                      )
    curs = conn.cursor()
    sql  = 'INSERT INTO redtable_raw_2022.AIR_PORTAL({}) VALUES(%s{});'.format(db_cols,",%s" * 14)
    val = list(map(lambda x : tuple(x), df_result.values))
    curs.executemany(sql,val)
    curs.close()
    conn.close()
    return 'DB 등록 완료'


save_to_AIR_PORTAL(og)

쿼리 완료 
쿼리 작동 시간 : 3.5800015926361084
